In [1]:
import pandas as pd
import random
import re
from IPython.display import clear_output
import itertools

In [2]:
#create an index file for the huge data file
def createIndex(filePath):
    file = open(filePath,'r');
    indexFilePath = filePath+"_index.csv"
    indexFile = open(indexFilePath,'w');
    indexFile.close()
    
    offset = 0
    lineNumber = 0
    for line in file:
#         clear_output(wait=True)
#         print(lineNumber)
        indexFile = open(indexFilePath,'a');
        indexFile.write(str(lineNumber) + "," + str(offset) + "\n")
        indexFile.close()
        lineNumber = lineNumber + 1
        offset += len(line)
    file.close()
    return indexFilePath

In [3]:
def generateNIndecesFrom(n, rangeOfIndeces):
    print("Generating " + str(n) + " indeces from range")
    allIndeces = random.sample(rangeOfIndeces, n)
    allIndeces = pd.Series(data = allIndeces)
    allIndeces = allIndeces.sort_values().reset_index().drop(['index'],axis=1)
    allIndeces.columns = ['Index'];
    return allIndeces

In [4]:
#Load or Generate the 2M indeces to use
try:
    twoMIndeces = pd.read_csv('2MIndeces.csv',squeeze = True)
    
except:
    print("There were not 2000000 data points")
    twoMIndeces = generateNIndecesFrom(2000000,range(0,45840617)) # this range is because there are this number of records in the training set.
    twoMIndeces.to_csv('2MIndeces.csv')

In [11]:
def generateSubSet(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers,frameSaveName):
    totalNumIterations = int(totalNumRows/numRowsPerItteration)
    print("Number of itterations = " + str(totalNumIterations))
    totalNumRowsTraversed = 0
    prevsize = 0
    for i in range(totalNumIterations + 1):
#         
#         print("Itteration number: " + str(i))
#         print("skipRows: " + str(i * numRowsPerItteration))
#         print("Read in : " + str(numRowsPerItteration))
        curData = pd.read_table(file,skiprows = i * numRowsPerItteration, nrows = numRowsPerItteration,header=None)
        curData.index = [i for i in range(i*numRowsPerItteration,i*numRowsPerItteration + curData.shape[0])]
        totalNumRowsTraversed = totalNumRowsTraversed + curData.shape[0]

#         print(curData.shape)
#         print(curData.index.shape)

        
        curData.columns = column_headers
        curData['Index'] = curData.index
        
        curIndexRange = indexValues['Index'][(indexValues['Index'] < (i*numRowsPerItteration + numRowsPerItteration)) & (indexValues['Index'] > (i*numRowsPerItteration))]
        curData = curData[curData['Index'].isin(curIndexRange)]

        dataFrame = pd.concat([dataFrame,curData])
        
        clear_output()
        print("Extraction Stats: " + str(dataFrame.shape[0]) + " percent: " + str(dataFrame.shape[0] / indexValues.shape[0] * 100) + "%")
        print("Document Stats: " + str(totalNumRowsTraversed) + " percent: " + str(totalNumRowsTraversed/totalNumRows*100) + "%")
        if (dataFrame.shape[0] - prevsize) > 500000:
            prevsize = dataFrame.shape[0]
            dataFrame.to_csv(frameSaveName)
        elif dataFrame.shape[0] == indexValues.shape[0]:
            print("Finished with the data collection")
            dataFrame.to_csv(frameSaveName)
    print("Extraction is Done, now saving frame")        
    dataFrame.to_csv(frameSaveName)
    return dataFrame

In [12]:
column_headers = pd.read_csv('data_columns.csv',squeeze = True)
allData = pd.DataFrame()
allData = generateSubSet('dac/train.txt',allData,twoMIndeces,4000000,46000000,column_headers,'2MDataFinalDesktop.csv')

Extraction Stats: 1999999 percent: 99.99995%
Document Stats: 45840617 percent: 99.6535152173913%
Extraction is Done, now saving frame


In [13]:
# column_headers = pd.read_csv('data_columns.csv',squeeze = True)
# allData = pd.DataFrame()

# numRowsPerItteration = 100000
# # numiterations = 0
# totalNumRows = 48000000
# totalNumIterations = int(totalNumRows/numRowsPerItteration)
# for i in range(totalNumIterations):
#     curData = pd.read_table('dac/train.txt',skiprows = i * numRowsPerItteration,nrows = numRowsPerItteration,header=None)
#     curData.index = [i for i in range(i*numRowsPerItteration,i*numRowsPerItteration + numRowsPerItteration)]
#     curData.columns = column_headers
#     curData['Index'] = curData.index
#     curIndexRange = twoMIndeces['Index'][(twoMIndeces['Index'] < (i*numRowsPerItteration + numRowsPerItteration)) & (twoMIndeces['Index'] > (i*numRowsPerItteration))]
#     curData = curData[curData['Index'].isin(curIndexRange)]
# #     print(allData.shape)
# #     print(curData.head())
#     allData = pd.concat([allData,curData])
#     clear_output(wait=True)
#     print(allData.shape)

In [14]:
allData.head()

Index,label,integer_1,integer_2,integer_3,integer_4,integer_5,integer_6,integer_7,integer_8,integer_9,...,categorical_18,categorical_19,categorical_20,categorical_21,categorical_22,categorical_23,categorical_24,categorical_25,categorical_26,Index
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN,4
14,0,0.0,51,84.0,4.0,3633.0,26.0,1.0,4.0,8.0,...,12195b22,21ddcdc9,b1252a9d,fa131867,NaN,dbb486d7,8ecc176a,e8b83407,c43c3f58,14
64,0,1.0,1,5.0,8.0,7.0,8.0,1.0,8.0,8.0,...,4b340164,NaN,NaN,NaN,NaN,3a171ecb,NaN,NaN,NaN,64
91,0,NaN,-1,NaN,NaN,23204.0,NaN,0.0,32.0,38.0,...,15a36060,NaN,NaN,0014c32a,NaN,423fab69,3b183c5c,NaN,NaN,91
117,1,NaN,-1,NaN,NaN,446.0,30.0,2.0,27.0,28.0,...,fffe2a63,21ddcdc9,b1252a9d,eb0fc6f8,NaN,32c7478e,df487a73,001f3601,c27f155b,117


In [28]:
allData.tail()

Index,label,integer_1,integer_2,integer_3,integer_4,integer_5,integer_6,integer_7,integer_8,integer_9,...,categorical_18,categorical_19,categorical_20,categorical_21,categorical_22,categorical_23,categorical_24,categorical_25,categorical_26,Index
45840529,0,NaN,417,57.0,1.0,11952.0,NaN,0.0,21.0,281.0,...,15a36060,NaN,NaN,0014c32a,NaN,bcdee96c,3b183c5c,NaN,NaN,45840529
45840572,0,NaN,0,1.0,1.0,18597.0,225.0,7.0,3.0,117.0,...,7e32f7a4,NaN,NaN,a4b7004c,NaN,3a171ecb,b34f3128,NaN,NaN,45840572
45840578,0,0.0,59,20.0,7.0,1945.0,120.0,7.0,36.0,119.0,...,9e4517be,3014a4b1,5840adea,572bdde8,NaN,55dd3565,9fa3e01a,001f3601,d9bcfc08,45840578
45840600,0,1.0,71,34.0,16.0,21.0,3.0,39.0,22.0,80.0,...,5bb2ec8e,d271d52f,b1252a9d,be2263d6,NaN,32c7478e,3fdb382b,e8b83407,49d68486,45840600
45840612,0,NaN,-1,NaN,NaN,34722.0,38.0,1.0,0.0,41.0,...,891589e7,1d1eb838,a458ea53,840e0f0f,78e2e389,bcdee96c,d16bdc4c,e8b83407,095e16c8,45840612


In [30]:
twoMIndeces['Index']

0                 4
1                14
2                64
3                91
4               117
5               130
6               168
7               186
8               204
9               235
10              250
11              353
12              384
13              403
14              423
15              430
16              444
17              447
18              509
19              527
20              532
21              544
22              551
23              557
24              574
25              578
26              582
27              589
28              616
29              634
             ...   
1999970    45840023
1999971    45840025
1999972    45840072
1999973    45840111
1999974    45840120
1999975    45840123
1999976    45840141
1999977    45840157
1999978    45840229
1999979    45840230
1999980    45840240
1999981    45840265
1999982    45840281
1999983    45840319
1999984    45840328
1999985    45840360
1999986    45840389
1999987    45840407
1999988    45840416
